In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [20]:
url2 = 'https://lyricstranslate.com/en/collection/chinese-songs-transliteration-easy-learning'
html_content2 = requests.get(url2).text
soup2 = BeautifulSoup(html_content2)

In [21]:
links_list = []
for i in soup2.find_all("span", class_="collection-title"):
    for c in i.children:
        links_list.append(c['href'])

In [36]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.model_selection import train_test_split

from bs4 import BeautifulSoup
import requests

import unicodedata
import re
import numpy as np
import os
import io
import time

//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1)

In [37]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFKD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r'([?.!,¿])', r' \1 ', w)
    w = re.sub(r'[""]', " ", w)
    
    # add space between chinese characters without affecting english letters
    w = re.sub(r'(?<=[^a-z\W\d_])(?=[^a-z\W\d_])', ' ', w)
    
    # replace everything with space except (a-z, A-Z, ".", "?", "!", ",")
#     w = re.sub(r'[^a-zA-Z?.,!¿]', ' ', w)
    
    w.rstrip().strip()
    
    # add a start and an end token to the sentence
    # so that the model know when to start and stop
    w = '<start> ' + w + ' <end>'
    return w




In [59]:
def create_dataset(url):
    html_content = requests.get(url).text
    soup = BeautifulSoup(html_content)
    lines = soup.find_all(class_="ltf")
    if len(lines) == 2:
        word_pairs = [[preprocess_sentence(w) for w in l.get_text().split('\n')] for l in lines]
#     return zip(*word_pairs)
        return word_pairs
    else:
        return [[], []]


In [78]:
zhall = []
trall = []

for lst in links_list:
    zh, tr = create_dataset("https://lyricstranslate.com/"+lst)
    zhall.append(zh)
    trall.append(tr)

In [64]:
print(trall[:10])

[['<start> ni cengjing dui wo shuoguo <end>', '<start> yongyuan de ai wo <end>', '<start> shui zhidao ni dehua er <end>', '<start> dou shi zai pian wo <end>', '<start> ni henxin paoqi wo <end>', '<start> ye buguan wo sihuo <end>', '<start> shui ai wo shui ai wo <end>', '<start> shui lai ai wo <end>', '<start> buzhi shui lai ai wo xiangqile xinli nanguo <end>', '<start> wo dui ni bucuo <end>', '<start> wo de xin weile ni <end>', '<start> meiyou fangxiaguo <end>', '<start> pa ni leng pa ni re <end>', '<start> pa ni ke pa ni e <end>', '<start> shui ai wo shui ai wo <end>', '<start> shui lai ai wo <end>', '<start> buzhi shui lai ai wo ni conglai meiyou gei wo <end>', '<start> yidian er kuaile <end>', '<start> wo ming li zhuding <end>', '<start> yao shou ni de zhemo <end>', '<start> ai shi jia qing shi suo <end>', '<start> wo pian yuan shou zhemo <end>', '<start> shui ai wo shui ai wo <end>', '<start> shui lai ai wo <end>', '<start> buzhi shui lai ai wo xiangqile xinli nanguo <end>', '<star

In [82]:
zhall_2 = []
for z in zhall:
#     zhall_2.append(z.split("<end>"))
    for i in z:
        zhall_2.append(i)
print(zhall_2[:50])

['<start> 你 曾 經 對 我 說 過 <end>', '<start> 永 遠 的 愛 我 <end>', '<start> 誰 知 道 你 的 話 兒 <end>', '<start> 都 是 在 騙 我 <end>', '<start> 你 狠 心 拋 棄 我 <end>', '<start> 也 不 管 我 死 活 <end>', '<start> 誰 愛 我 誰 愛 我 <end>', '<start> 誰 來 愛 我 <end>', '<start> 不 知 誰 來 愛 我 想 起 了 心 裡 難 過 <end>', '<start> 我 對 你 不 錯 <end>', '<start> 我 的 心 為 了 你 <end>', '<start> 沒 有 放 下 過 <end>', '<start> 怕 你 冷 怕 你 熱 <end>', '<start> 怕 你 渴 怕 你 餓 <end>', '<start> 誰 愛 我 誰 愛 我 <end>', '<start> 誰 來 愛 我 <end>', '<start> 不 知 誰 來 愛 我 你 從 來 沒 有 給 我 <end>', '<start> 一 點 兒 快 樂 <end>', '<start> 我 命 裡 注 定 <end>', '<start> 要 受 你 的 折 磨 <end>', '<start> 愛 是 枷 情 是 鎖 <end>', '<start> 我 偏 願 受 折 磨 <end>', '<start> 誰 愛 我 誰 愛 我 <end>', '<start> 誰 來 愛 我 <end>', '<start> 不 知 誰 來 愛 我 想 起 了 心 裡 難 過 <end>', '<start> 我 對 你 不 錯 <end>', '<start> 我 的 心 為 了 你 <end>', '<start> 沒 有 放 下 過 <end>', '<start> 怕 你 冷 怕 你 熱 <end>', '<start> 怕 你 渴 怕 你 餓 <end>', '<start> 誰 愛 我 誰 愛 我 <end>', '<start> 誰 來 愛 我 <end>', '<start> 不 知 誰 來 愛 我 你 曾 經 對 我 說 過 <end>', '<start> 永 遠 的

In [85]:
def work(ll):
    
    res = list()
    for l in ll:
        for i in l:
            res.append(i)
    return res

In [87]:
zhall2 = work(zhall)
trall2 = work(trall)


In [89]:
pd.DataFrame({"ZH":zhall2[:500], "EN":trall2[:500]})

,ZH,EN
0,<start> 你 曾 經 對 我 說 過 <end>,<start> ni cengjing dui wo shuoguo <end>
1,<start> 永 遠 的 愛 我 <end>,<start> yongyuan de ai wo <end>
2,<start> 誰 知 道 你 的 話 兒 <end>,<start> shui zhidao ni dehua er <end>
3,<start> 都 是 在 騙 我 <end>,<start> dou shi zai pian wo <end>
4,<start> 你 狠 心 拋 棄 我 <end>,<start> ni henxin paoqi wo <end>
...,...,...
495,<start> 才 算 魔 鬼 的 身 材 <end>,<start> cai suan mo gui de shen cai <end>
496,<start> 不 愿 为 男 人 而 忍 耐 <end>,<start> bu yuan wei nan ren er ren nai bu gua...
497,<start> [x2] 做 回 自 己 才 是 美 丽 <end>,<start> bu guan mei li de chen ai <end>
498,<start> 不 管 别 人 怎 么 说 你 <end>,<start> zuo hui zi ji ke bu ke yi <end>
